In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import sklearn as skl
import matplotlib.pyplot as plt

In [ ]:
data=pd.read_csv('actu_finance.csv')
data.head()
data.describe()


In [ ]:
data['Unnamed: 0']=pd.to_datetime(data['Unnamed: 0'])
data.sort_values('Unnamed: 0')
data.set_index('Unnamed: 0',inplace=True)
data.head()

In [ ]:
sns.pairplot(data, hue='Unnamed: 0')

## on retient 5n actifs

In [ ]:
data.info()

In [ ]:
data.replace("NR", np.nan, inplace=True)
print("Pourcentages indisponibles par colones")

print(100*data[[col for col in data.columns if col.endswith('_ACTU')]].isna().sum()/data.shape[0])

print("Nombre d'actu indisponibles au total")
print(data.isna().sum().sum())

In [ ]:
# dic={}
# for col in data.columns[:8]:
#   if col.endswith('_ACTU'):
#     dic[f"{col.split('_')[0]}"]=data[[f"{col}", f"{col.split('_')[0]}_RDMT_5"]].dropna()

dic={}
for col in data.columns[:8]:
  if col.endswith('_ACTU'):
    dic[f"{col.split('_')[0]}"]=data[[f"{col}", f"{col.split('_')[0]}_RDMT_5"]].dropna()
    dic[f"{col.split('_')[0]}"].columns=['ACTU', 'RDMT']

for key, value in dic.items():
  print('Statistiques descriptives pour ',key)
  print(value.describe(include=[object]))
  print(value.describe(exclude=[object]))
  print('-----------------------------------------------------------------------'*2)
  print('\n')


In [ ]:
#dic['BNP']
rest=pd.concat([dic['BNP'], dic['AIR']])
rest.dropna().head()

dic['BNP'].iloc[:,1:2].groupby(dic['BNP'].index.year).count()

for key,value in dic.items():
  print("Nombre d'actualité par année")
  print(value.iloc[:,1:2].groupby(value.index.year).count())
  print('-'*100)
  print('\n')

In [ ]:
tab=[value for key, value in dic.items()]
df=pd.concat(tab, axis=0).reset_index()
df.rename(columns={'Unnamed: 0':'Date'}, inplace=True)
df.set_index('Date', inplace=True)
#df.dropna().head()
df.head()

In [ ]:
q20=df['RDMT'].quantile(0.2)
print(q20)
df['Y']=df['RDMT'].apply(lambda x: 1 if x<q20 else 0)
print(df.head())

In [ ]:
from sentence_transformers import SentenceTransformer
model=SentenceTransformer("all-MiniLM-L6-v2")

#embeddings=model.encode()
df['ACTU_EMB'] = df['ACTU'].astype(str).apply(lambda x: model.encode(x))
df.head()

In [ ]:
import numpy as np

# Extraction des features X et de la cible Y
X = np.stack(df['ACTU_EMB'].values)
y = df['Y'].values

df_train = df[df.index.year < 2023]
df_test = df[df.index.year == 2023]

X_train = np.stack(df_train['ACTU_EMB'].values)
y_train = df_train['Y'].values

X_test = np.stack(df_test['ACTU_EMB'].values)
y_test = df_test['Y'].values


## Neural network

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam

model = Sequential([
    Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
    Dropout(0.2),
    Dense(64, activation='relu'),
    Dense(1, activation='sigmoid')  # binaire
])

model.compile(optimizer=Adam(0.001), loss='binary_crossentropy', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=20, batch_size=32, validation_split=0.2)

In [ ]:
from sklearn.metrics import f1_score, precision_score, recall_score

y_pred_prob = model.predict(X_test)
y_pred = (y_pred_prob > 0.5).astype(int)

print("F1 Score :", f1_score(y_test, y_pred))
print("Précision :", precision_score(y_test, y_pred))
print("Recall :", recall_score(y_test, y_pred))


In [ ]:
df.to_csv('actu_finance_emb.csv')